# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.
cpu is available.


# 1. set param from Front End

In [2]:
# regression/forecasting model parameter samples
GRU_rg_model_info = {'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': 'True'}
LSTM_rg_model_info = {'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': 'True'}
CNN_1D_rg_model_info = {'output_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 0, 'dropout': 0.1}
LSTM_FCNs_rg_model_info = {'num_layers': 2, 'lstm_dropout': 0.4, 'fc_dropout': 0.1}

# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'GRU_rg',   # 'GRU_rg', 'LSTM_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg'
        'model_name' : 'None',
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : GRU_rg_model_info
    }
}

# "bucket_name": 'integration',
# "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
# "feature_list":['Temperature', 'out_temp','sin_hour']

# "bucket_name": 'integration',
# "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
# "feature_list":['Temperature']

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'step_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':24, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'GRU_rg',    # 'GRU_rg', 'LSTM_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg'
        'model_name' : 'None',
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : GRU_rg_model_info
    }
}

# classification model parameter samples
GRU_cf_model_info = {'hidden_size': 64, 'num_layers':2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': 'True', 'num_classes': 6, 'rnn_type': 'gru'}
LSTM_cf_model_info = {'hidden_size': 64, 'num_layers':2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': 'True', 'num_classes': 6, 'rnn_type': 'lstm'}
CNN_1D_cf_model_info = {'output_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 0, 'dropout': 0.1, 'num_classes': 6}
LSTM_FCNs_cf_model_info = {'num_layers': 2, 'lstm_dropout': 0.4, 'fc_dropout': 0.1, 'num_classes': 6}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'classification',
        'model_method' : 'GRU_cf',  # 'GRU_cf', 'LSTM_cf', 'CNN_1D_cf', 'LSTM_FCNs_cf'
        'model_name' : 'None',
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : GRU_cf_model_info
    }
}

# 2. Backend

In [3]:
# 1. Backend Prameter Setting
params = params1

# parameter tunning
params = ML_api.convert_param_for_backend(params)

In [4]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (params['ingestion_param_X'], mongodb_client)
params['train_data_info'] = train_data_info

In [5]:
# 3. Data Preparation
train_X_array, train_y_array, val_X_array, val_y_array = ML_api.train_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
./scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl
['value']
Make New scaler File
{'split_mode': 'window_split', 'data_clean_option': False, 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5, 'future_step': None, 'past_step': 144}
window_split
None
window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2

In [6]:
print(train_X_array.shape)
print(train_y_array.shape)

(72, 144, 24)
(72, 1)


In [7]:
# 4.Training
params = ML_api.ML_train(params, train_X_array,  train_y_array, val_X_array, val_y_array)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/GRU_rg/regression_energy_cleanLevel4_trainX_regression_GRU_rg_/regression_energy_cleanLevel4_trainX/model.pkl
Start training model
[1/5] Training loss: 0.2152	 Validation loss: 0.1540
[2/5] Training loss: 0.1981	 Validation loss: 0.1418
[3/5] Training loss: 0.1745	 Validation loss: 0.1300
[4/5] Training loss: 0.1777	 Validation loss: 0.1185
[5/5] Training loss: 0.1632	 Validation loss: 0.1072

Training complete in 0m 6s


In [8]:
params

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},


In [9]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "./meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, params)

======== OK ========


200